In [1]:
#NOTE: use paimg9 env
import sys
import os
import numpy as np
import openslide
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')
import pandas as pd
import warnings
import torch
import torch.nn as nn

from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader
import torch.optim as optim
from pathlib import Path

sys.path.insert(0, '../Utils/')
from Utils import create_dir_if_not_exists
from Utils import generate_deepzoom_tiles, extract_tile_start_end_coords, get_map_startend
from Utils import get_downsample_factor
from Utils import minmax_normalize, count_label
from Utils import log_message, set_seed
from Eval import compute_performance, plot_LOSS, compute_performance_each_label, get_attention_and_tileinfo
from train_utils import pull_tiles, get_feature_label_array_dynamic
from train_utils import ModelReadyData_diffdim, convert_to_dict, prediction
from Model import Mutation_MIL_MT
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
####################################
######      USERINPUT       ########
####################################
SELECTED_LABEL = ["AR","MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2","PTEN","RB1","TP53","TMB_HIGHorINTERMEDITATE","MSI_POS"]
SELECTED_FEATURE = [str(i) for i in range(0,2048)] + ['TUMOR_PIXEL_PERC']
TUMOR_FRAC_THRES = 0.9
TRAIN_SAMPLE_SIZE = "ALLTUMORTILES"
TRAIN_OVERLAP = 100
TEST_OVERLAP = 0
SELECTED_FOLD = 0
feature_extraction_method = 'retccl'
selected_ids = ['OPX_207', 'OPX_208', 'OPX_209', 'OPX_210', 'OPX_211', 'OPX_212', 'OPX_213', 'OPX_214', 'OPX_215', 'OPX_216']
print(selected_ids)

##################
###### DIR  ######
##################
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
id_path = proj_dir + '/data/OPX/'
test_tile_path =  proj_dir + 'intermediate_data/cancer_prediction_results110224/IMSIZE250_OL' + str(TEST_OVERLAP) + '/'


################################################
#Create output dir
################################################
outdir =   proj_dir + 'intermediate_data/model_ready_data/feature_' + feature_extraction_method + '/MAXSS'+ str(TRAIN_SAMPLE_SIZE)  + '_TrainOL' + str(TRAIN_OVERLAP) +  '_TestOL' + str(TEST_OVERLAP) + '_TFT' + str(TUMOR_FRAC_THRES) + '/'
create_dir_if_not_exists(outdir)
outdir2 =  outdir + "/split_fold" + str(SELECTED_FOLD) + "/" + "init_data_info/" 
create_dir_if_not_exists(outdir2)

##################
#Select GPU
##################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
set_seed(0)

['OPX_207', 'OPX_208', 'OPX_209', 'OPX_210', 'OPX_211', 'OPX_212', 'OPX_213', 'OPX_214', 'OPX_215', 'OPX_216']
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/model_ready_data/feature_retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/model_ready_data/feature_retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9//split_fold0/init_data_info/' already exists.
cuda:0


In [3]:
############################################################################################################
#Get features and labels
#NOTE: OPX_005 has no tumor tiles in fold0 train, so excluded in this step
############################################################################################################
feature_name = 'features_alltiles_' +  feature_extraction_method 
test_feature, test_label, test_info, test_tf_info, select_test_ids = get_feature_label_array_dynamic(test_tile_path, feature_name, selected_ids, SELECTED_LABEL, SELECTED_FEATURE, "Test", tumor_fraction_thres = TUMOR_FRAC_THRES)


torch.save(test_feature,  outdir2 + 'newMSI_test_feature.pth')
torch.save(test_label,  outdir2 + 'newMSI_test_label.pth')
torch.save(test_info,   outdir2 + 'newMSI_test_info.pth')
torch.save(test_tf_info,   outdir2 + 'newMSI_test_tf_info.pth')
torch.save(select_test_ids,   outdir2 + 'newMSI_test_ids.pth')


0


In [4]:
############################################################################################################
#Count Distribution
############################################################################################################
test_counts = count_label(test_label, SELECTED_LABEL, "newMSI_Test")
test_counts.to_csv(outdir2 + "newSMI_Test_counts.csv")